# Custom agent

不管是任意的大模型，我们都可以根据大模型的特性来打造智能体。这通常是创建代理的最可靠方法。

我们将首先在没有历史记录的情况下创建它，但随后我们将展示如何添加历史记录。需要历史记录才能启用对话。

## 加载LLM

首先，让我们加载将用于控制代理的语言模型。

In [74]:
from langchain_openai import ChatOpenAI
import os

api_key = os.getenv("LOCAL_API_KEY")
base_url = os.getenv("LOCAL_API_BASE")

model = ChatOpenAI(api_key=api_key, 
                   base_url=base_url,
                   temperature=0.3, 
                   max_tokens=4096, 
                )

## 定义工具

接下来，让我们定义一些要使用的工具。让我们编写一个非常简单的 Python 函数来计算传入的单词的长度。

请注意，这里我们使用的函数文档字符串非常重要。

In [75]:
from langchain.agents import tool

@tool
def word_length(word: str) -> int:
    """计算传入的单词的长度"""
    return len(word)

word_length.invoke("hello")

5

In [76]:
import requests

@tool
def get_weather(location: str) -> dict:
    """根据城市获取天气数据"""
    api_key = "SKcA5FGgmLvN7faJi"
    city_codes = {
        "西安": "xi'an",
        "北京": "beijing",
        "上海": "shanghai",
        "广州": "guangzhou",
        "深圳": "shenzhen",
        "杭州": "hangzhou",
        "成都": "chengdu",
        "武汉": "wuhan"
    }
    
    # 处理城市名称
    location = location.replace("市", "").replace("区", "")
    location = city_codes.get(location, location)  # 如果有映射就用映射，没有就用原名
    
    
    url = f"https://api.seniverse.com/v3/weather/now.json?key={api_key}&location={location}&language=zh-Hans&unit=c"
    try:
        response = requests.get(url)
        #print(location)
        if response.status_code == 200:
            data = response.json()
            #print(data)
            weather = {
                'description':data['results'][0]["now"]["text"],
                'temperature':data['results'][0]["now"]["temperature"]
            }
            return weather
        else:
            raise Exception(f"失败接收天气信息：{response.status_code}")
    except Exception as e:
        return {"description": "获取天气信息失败", "detail": str(e)}
            
get_weather.invoke("广州")

{'description': '多云', 'temperature': '14'}

In [77]:
import requests
import json
 
@tool
def searxng_search(query):
    """输入搜索内容，使用SearXNG进行搜索"""
    SEARXNG_URL = "http://127.0.0.1:6699"
    SEARXNG_API_KEY = "1234567890"
    params = {}
    # 设置搜索参数
    params["q"] = query
    params["format"] = "json"
    params["token"] = SEARXNG_API_KEY
    # 发送GET请求
    response = requests.get(SEARXNG_URL, params=params)
    # return response.text
    
    # 检查响应状态码
    if response.status_code == 200:
        res = response.json()
        resList = []
        for item in res['results']:
            resList.append(
                {
                    'title':item['title'],
                    'content':item['content'],
                    'url':item['url']
                }
            )
            if len(resList) > 3:
                break
        return resList
    else:
        response.raise_for_status()

search = searxng_search.invoke("今天西安天气如何？")
search

[{'title': '西安天气预报,西安7天天气预报,西安15天天气预报,西安天气查询',
  'content': '今明天（12月17日至15日），弱冷空气将影响中东部地区；19日至21日，又有较强冷空气东移南下。 今天（12月16日），我国大部气温回升，但昼夜温差将拉大至10℃至15℃，公众请适时调整着装，谨防感冒。 17日之后，两轮冷空气影响我国。 今明天全国大部地区多晴朗干燥天气，未来一段时间，还将有冷空气补充南下，南方昼夜温差加大，多地较常年同期偏冷明显。 今天起至17日，中东部大部将开启一段降水稀少的时期。 与此同时，冷空气源源不断南下，南方多地气温还将刷新下半年来新低。 今后三天（12月13至15日），较强冷空气继续影响我国，多地将刷新立冬以来的气温新低。 同时，今天我国北方降雪基本结束。 今天我国中东部雨雪范围较大，明天起雨雪将快速缩减，之后全国将迎来大范围晴或多云天气。',
  'url': 'https://www.weather.com.cn/weather/101110101.shtml'},
 {'title': '8 - song and lyrics by Billie Eilish | Spotify',
  'content': 'March 29, 2019 - Listen to 8 on Spotify. Billie Eilish · Song · 2019.',
  'url': 'https://open.spotify.com/track/6X29iaaazwho3ab7GNue5r'},
 {'title': '5 - 维基百科，自由的百科全书',
  'content': 'July 19, 2024 - 5（五）是4与6之间的自然数，是第3个质数。 ... 在十进制中，它是唯一一个以数字5为个位数的素数，其他以数字5为个位数的数均为5的倍数（注意10是5的倍数，因此对于任何整数n， ... 此数字虽然是自然质数，但不是高斯...',
  'url': 'https://zh.wikipedia.org/zh-hans/5'},
 {'title': '西安 - 中国气象局-天气预报-城市预报',
  'content': '时间, 14:00, 17:00, 20:00, 23:00, 02:00, 05:0

In [78]:
from langchain_community.utilities import SearxSearchWrapper

search = SearxSearchWrapper(
    searx_host="http://127.0.0.1:6699",
    k=5
)
search.run("刘亦菲")

'刘亦菲，1987年8月25日出生于湖北省武汉市，华语影视女演员、歌手，毕业于北京电影学院表演系。2002年，主演个人首部电视剧《金粉世家》白秀珠一角踏入演艺圈。2003年，凭借金庸武侠剧《天龙八部》女主王语嫣一角崭露头角。2004年，凭借仙侠剧《仙剑奇侠传》赵灵儿一角获得高人气与关注度 ...\n\nIt\'s a pleasure to be back in Paris immersing myself in the world of inspiration of Nicolas Ghesquière, Louis Vuitton Women\'s Collection Artistic Director.\n\n66K Followers, 54 Following, 1429 Posts - 刘亦菲Liu Yifei / CrystalLiu (@fenglixi_) on Instagram: "⭐ #LiuYifei @yifei_cc \u200d♀️ #Mulan #TheChineseWidow ...\n\nLiu Yifei (刘亦菲; born August 25, 1987) is a Chinese-born American [a] actress. [3] She has appeared multiple times on Forbes China Celebrity 100 list and was named one of the New Four Dan actresses of China by Tencent Entertainment in 2009. [4] She is known for her roles in Chinese TV shows such as The Story of a Noble Family, Demi-Gods and Semi-Devils, Chinese Paladin, A Dream of ...\n\nMay 11, 2023 - 刘亦菲 · 粉丝6978.8万 关注225 · 关注 私信 · V指数 文娱 78.05分 · 昨日阅读人数100万+，互动数23万 · 演员刘亦菲 · 精选 · 微博 · 视频 · 相册 · 超话 · 全部微博（1295） · 刘亦菲 · 8-2

In [79]:
tools = [word_length,get_weather,searxng_search]

## 创建提示

现在让我们创建提示。由于 OpenAI 函数调用针对工具使用进行了微调，因此我们几乎不需要任何关于如何推理或如何输出格式的说明。我们将只有两个输入变量： input 和 agent_scratchpad 。 input 应为包含用户目标的字符串。 agent_scratchpad 应该是包含先前代理工具调用和相应工具输出的消息序列。

In [80]:
promptTemplate = """尽可能的帮助用户回答任何问题。
您可以使用以下工具来帮忙解决问题，如果已经知道答案，也可以直接回答：
{tools}

回复格式说明：
----------------------------------

回复我时，请以以下两种格式之一输出问题：

选项1：如果您希望人类使用工具，请使用此选项。
采用以下JSON模式格式化的回复内容：
```json
{{
    "reason": string, \\ 叙述使用工具的原因
    "action": string, \\ 要使用的工具。 必须是 {tool_names} 之一
    "action_input": string \\ 工具的输入
}}
```

选项2：如果您认为你自己已经有了答案或者已经通过使用工具找到了答案，想直接对人类做出反应，请使用此选项。
采用以下JSON模式格式化的回复内容：
```json
{{
    "action": "Final Answer",
    "answer": string \\ 最终回复问题的答案放在这里！
}}
```

用户输入
----------------------------------
这是用户的输入（请记住通过单个选项，以JSON模式格式化的回复内容，不要回复其他内容）：
{input}
"""
promptTemplate

'尽可能的帮助用户回答任何问题。\n您可以使用以下工具来帮忙解决问题，如果已经知道答案，也可以直接回答：\n{tools}\n\n回复格式说明：\n----------------------------------\n\n回复我时，请以以下两种格式之一输出问题：\n\n选项1：如果您希望人类使用工具，请使用此选项。\n采用以下JSON模式格式化的回复内容：\n```json\n{{\n    "reason": string, \\ 叙述使用工具的原因\n    "action": string, \\ 要使用的工具。 必须是 {tool_names} 之一\n    "action_input": string \\ 工具的输入\n}}\n```\n\n选项2：如果您认为你自己已经有了答案或者已经通过使用工具找到了答案，想直接对人类做出反应，请使用此选项。\n采用以下JSON模式格式化的回复内容：\n```json\n{{\n    "action": "Final Answer",\n    "answer": string \\ 最终回复问题的答案放在这里！\n}}\n```\n\n用户输入\n----------------------------------\n这是用户的输入（请记住通过单个选项，以JSON模式格式化的回复内容，不要回复其他内容）：\n{input}\n'

In [81]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "你是非常强大的助手，你可以使用各种工具来完成人类交给的问题和任务。"),
    ("user", promptTemplate),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])
prompt


ChatPromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMess

In [82]:
from langchain.tools.render import render_text_description

prompt = prompt.partial(
    tools=render_text_description(list(tools)),
    tool_names=", ".join([t.name for t in tools])
)
prompt


ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Ann

In [83]:
render_text_description(list(tools))

'word_length(word: str) -> int - 计算传入的单词的长度\nget_weather(location: str) -> dict - 根据城市获取天气数据\nsearxng_search(query) - 输入搜索内容，使用SearXNG进行搜索'

## 将工具绑定到LLM

在这种情况下，我们依赖于 OpenAI 工具调用 LLMs，它将工具作为一个单独的参数，并经过专门训练，知道何时调用这些工具。

要将我们的工具传递给代理，我们只需要将它们格式化为 OpenAI 工具格式并将它们传递给我们的模型。（通过 bind 对函数进行 -ing，我们确保每次调用模型时都传入它们。

## 创建代理

将这些部分放在一起，我们现在可以创建代理。我们将导入最后两个实用程序函数：一个用于格式化中间步骤（代理操作、工具输出对）以输入可发送到模型的消息的组件，以及一个用于将输出消息转换为代理操作/代理完成的组件。

In [84]:
from langchain.agents.json_chat.prompt import TEMPLATE_TOOL_RESPONSE

TEMPLATE_TOOL_RESPONSE

"TOOL RESPONSE: \n---------------------\n{observation}\n\nUSER'S INPUT\n--------------------\n\nOkay, so what is the response to my last comment? If using information obtained from the tools you must mention it explicitly without mentioning the tool names - I have forgotten all TOOL RESPONSES! Remember to respond with a markdown code snippet of a json blob with a single action, and NOTHING else - even if you just want to respond to the user. Do NOT respond with anything except a JSON snippet no matter what!"

In [85]:
TEMPLATE_TOOL_RESPONSE = """工具响应：
----------------------------------

{observation}

用户的输入：
----------------------------------
请根据工具的响应判断，是否能够回答问题：

{input}

请根据工具响应的内容，思考接下来的回复。回复格式严格按照前面所说的2种JSON回复格式，
选择其中1种进行回复。请记住通过单个选项，以JSON模式格式化回复内容，不要回复其他内容。
"""


In [86]:
from langchain_core.messages import HumanMessage, BaseMessage, AIMessage
from typing import List


def format_log_to_messages(
    query,
    intermediate_steps,
    template_tool_response
):
    """Custruct the scratchpad that lets the agent continue its thought process"""
    thoughts: List[BaseMessage] = []
    for action, observation in intermediate_steps:
        thoughts.append(AIMessage(content=action.log))
        human_message = HumanMessage(
            content=template_tool_response.format(
                input=query,
                observation=observation
            )
        )
        thoughts.append(human_message)
    return thoughts

In [87]:
from langchain.agents.agent import AgentOutputParser
from langchain_core.output_parsers.json import parse_json_markdown
from langchain_core.exceptions import OutputParserException
from langchain_core.agents import AgentAction, AgentFinish
import logging

logger = logging.getLogger(__name__)

class JSONAgentOutputParser(AgentOutputParser):
    """Parses tool invocations and final answers in JSON format.

    Expects output to be in one of two formats.

    If the output signals that an action should be taken,
    should be in the below format. This will result in an AgentAction
    being returned.

    ```
    {
      "action": "search",
      "action_input": "2+2"
    }
    ```

    If the output signals that a final answer should be given,
    should be in the below format. This will result in an AgentFinish
    being returned.

    ```
    {
      "action": "Final Answer",
      "answer": "4"
    }
    ```
    """
    
    def parse(self, text):
        try:
            response = parse_json_markdown(text)
            if isinstance(response, list):
                logger.warning(f"Got mutiple action responses: {response}")
                response = response[0]
            if response.get("action") == "Final Answer":
                return AgentFinish(
                    {"output": response.get("answer")},text
                )
            else:
                return AgentAction(
                    response['action'],
                    response.get("action_input", {}),
                    text
                )
        except OutputParserException as e:
            raise OutputParserException(f"Failed to parse tool call or final answer: {text}") from e
        
    @property
    def _type(self) -> str:
        return "json-agent"
    

In [88]:
from langchain_core.runnables import RunnablePassthrough

agent = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_log_to_messages(
            x["input"],
            x["intermediate_steps"],
            template_tool_response=TEMPLATE_TOOL_RESPONSE
        )
    )
    | prompt
    | model
    | JSONAgentOutputParser()
)

In [93]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)
agent_executor.invoke({"input": "今天汉中天气如何？"})



> Entering new AgentExecutor chain...
```json
{
    "reason": "获取今天的汉中的天气数据",
    "action": "get_weather",
    "action_input": "Hanzhong"
}
```{'description': '阴', 'temperature': '3'}```json
{
    "action": "Final Answer",
    "answer": "今天汉中天气阴沉，温度为3摄氏度"
}
```

> Finished chain.


{'input': '今天汉中天气如何？', 'output': '今天汉中天气阴沉，温度为3摄氏度'}

In [91]:
agent_executor.invoke({"input": "刘德华的老婆是谁？"})



> Entering new AgentExecutor chain...
```json
{
    "reason": "搜索有关刘德华的信息",
    "action": "searxng_search",
    "action_input": "刘德华的妻子"
}
```[{'title': '朱丽蒨_百度百科', 'content': '朱丽蒨（Carol），1966年4月6日出生于马来西亚槟城，祖籍福建诏安，马来西亚选美小姐、平面模特，刘德华的妻子。 朱丽倩自小随家人移居吉隆坡。 1984年，参加马来西亚槟城"新潮小姐"选美获得季军，同年成为时尚杂志《新潮》七月封面小姐。', 'url': 'https://baike.baidu.com/item/%E6%9C%B1%E4%B8%BD%E8%92%A8/63815079'}, {'title': 'Entertainment News, Celebrity News, Celebrity Gossip - E! Online', 'content': 'Your source for entertainment news, celebrities, celeb news, and celebrity gossip. Check out the hottest fashion, photos, movies and TV shows!', 'url': 'https://www.eonline.com/'}, {'title': '刘德华老婆朱丽倩露面，58岁臃肿显老态，夫唱妇随陪老公开 ...', 'content': '2024年11月4日 — 她，就是朱丽倩——刘德华的妻子，一位曾经的马来西亚选美冠军，如今却在媒体口中被描述为“58岁显老态”。当这位低调的天王嫂步入台北小巨蛋，她的每一个动作都 ...', 'url': 'https://www.sohu.com/a/823445775_121657392'}, {'title': 'E! News - YouTube', 'content': 'September 13, 2023 - E! News is your go-to source for breaking celebrity news & entertainment. E! N

{'input': '刘德华的老婆是谁？', 'output': '刘德华的老婆是朱丽蒨（Carol），一位马来西亚选美小姐和平面模特。'}

In [92]:
agent_executor.invoke({"input": "奥巴马的外祖父的亲舅舅的外祖母是谁？"})



> Entering new AgentExecutor chain...
```json
{
    "reason": "搜索奥巴马家族树",
    "action": "searxng_search",
    "action_input": "奥巴马的外祖父的亲舅舅的外祖母"
}
```[{'title': '从哈莱姆到广东——寻根的加勒比华裔黑人', 'content': '2018年2月7日 — 葆拉·麦迪逊(Paula Madison)曾经总是遐想，她的华裔亲人如今是什么样子。这位65岁的电视台退休高管在纽约市哈莱姆区长大。那里是黑人聚居区，她的母亲尼 ...', 'url': 'https://cn.nytimes.com/china/20180207/jamaica-china/'}, {'title': 'Entertainment News, Celebrity News, Celebrity Gossip - E! Online', 'content': 'Your source for entertainment news, celebrities, celeb news, and celebrity gossip. Check out the hottest fashion, photos, movies and TV shows!', 'url': 'https://www.eonline.com/'}, {'title': '无标题文档', 'content': '其实辜妈妈最显赫是她的出身。祖父是福州士大夫严复，盛宣怀家的五小姐嫁给了日据时代台湾首富板桥林家辜妈妈亲舅舅林熊征。辜妈妈在上海圣约翰大学念书时，曾给舅妈盛家五 ...', 'url': 'http://www.chinatoday.com.cn/china/2008/0812/p52.htm'}, {'title': 'E! News - YouTube', 'content': 'September 13, 2023 - E! News is your go-to source for breaking celebrity news & entertainment. E! News is the modern-and-bold mood for the pop c

{'input': '奥巴马的外祖父的亲舅舅的外祖母是谁？', 'output': '奥巴马的外祖父的亲舅舅的外祖母是盛宣怀家的五小姐。'}

## 添加历史记录

这太棒了 - 我们有一个代理！但是，此代理是无状态的 - 它不记得有关先前交互的任何信息。这意味着你不能轻易地提出后续问题。让我们通过添加内存来解决这个问题。

为此，我们需要做两件事：

1. 在提示符中为内存变量添加一个位置
2. 跟踪聊天记录

首先，让我们在提示中添加一个内存位置。为此，我们通过为带有 键 "chat_history" 的消息添加一个占位符 .请注意，我们将其放在新用户输入的上方（以遵循对话流）。

In [94]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "你是非常强大的助手，你可以使用各种工具来完成人类交给的问题和任务。"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", promptTemplate),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input', 'tool_names', 'tools'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag

In [96]:
chat_history = []

prompt = prompt.partial(
    tools=render_text_description(list(tools)),
    tool_names=", ".join([t.name for t in tools])
)
prompt


ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')]

In [97]:
agent = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_log_to_messages(
            x["input"],
            x["intermediate_steps"],
            template_tool_response=TEMPLATE_TOOL_RESPONSE
        )
    )
    | prompt
    | model
    | JSONAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [98]:
chat_history.extend(
    [
        HumanMessage(content="你好，我是老k。"),
        
        AIMessage(content="你好，老k，很高兴认识你！")
    ]
)

agent_executor.invoke({"input": "我叫什么？", "chat_history": chat_history})



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "answer": "你叫老k。"
}
```

> Finished chain.


{'input': '我叫什么？',
 'chat_history': [HumanMessage(content='你好，我是老k。', additional_kwargs={}, response_metadata={}),
  AIMessage(content='你好，老k，很高兴认识你！', additional_kwargs={}, response_metadata={})],
 'output': '你叫老k。'}

In [99]:
agent_executor.invoke({"input": "刘德华的老婆是谁？", "chat_history": chat_history})



> Entering new AgentExecutor chain...
```json
{
    "reason": "查找有关刘德华妻子的信息",
    "action": "searxng_search",
    "action_input": "刘德华的妻子"
}
```[{'title': '朱丽蒨_百度百科', 'content': '朱丽蒨（Carol），1966年4月6日出生于马来西亚槟城，祖籍福建诏安，马来西亚选美小姐、平面模特，刘德华的妻子。 朱丽倩自小随家人移居吉隆坡。 1984年，参加马来西亚槟城“新潮小姐”选美获得季军，同年成为时尚杂志《新潮》七月封面小姐。', 'url': 'https://baike.baidu.com/item/%E6%9C%B1%E4%B8%BD%E8%92%A8/63815079'}, {'title': 'Entertainment News, Celebrity News, Celebrity Gossip - E! Online', 'content': 'Your source for entertainment news, celebrities, celeb news, and celebrity gossip. Check out the hottest fashion, photos, movies and TV shows!', 'url': 'https://www.eonline.com/'}, {'title': '刘德华老婆朱丽倩露面，58岁臃肿显老态，夫唱妇随陪老公开 ...', 'content': '2024年11月4日 — 她，就是朱丽倩——刘德华的妻子，一位曾经的马来西亚选美冠军，如今却在媒体口中被描述为“58岁显老态”。当这位低调的天王嫂步入台北小巨蛋，她的每一个动作都 ...', 'url': 'https://www.sohu.com/a/823445775_121657392'}, {'title': 'E! News - YouTube', 'content': 'September 13, 2023 - E! News is your go-to source for breaking celebrity news & entertainment. E!

{'input': '刘德华的老婆是谁？',
 'chat_history': [HumanMessage(content='你好，我是老k。', additional_kwargs={}, response_metadata={}),
  AIMessage(content='你好，老k，很高兴认识你！', additional_kwargs={}, response_metadata={})],
 'output': '刘德华的老婆是朱丽蒨（Carol），一位马来西亚选美小姐和模特儿。'}

In [100]:
chat_history.extend(
    [
        HumanMessage(content="刘德华的老婆是谁？"),
        AIMessage(content="刘德华的老婆是朱丽倩。")
    ]
)
agent_executor.invoke({"input": "刘德华的老婆演过电影吗？", "chat_history": chat_history})



> Entering new AgentExecutor chain...
```json
{
    "reason": "搜索有关朱丽倩的信息",
    "action": "searxng_search",
    "action_input": "朱丽倩 演员 电影"
}
```[{'title': '朱丽蒨_百度百科', 'content': '1984年，18岁的朱丽蒨再次参加《新潮》封面小姐竞选，当年7月，她击败对手，以5369票成为"七月封面小姐"，更在1985年夺得"新潮小姐"季军兼青春小姐称号。 参加选美后，朱丽蒨完成高中学业，未继续求学 [2]。 1985年，朱丽蒨以5369票当选 马来西亚 时尚杂志《新潮》封面小姐 [1]。 1986年，朱丽蒨结识随 香港明星足球队 赴吉隆坡的 刘德华。 1989年，朱丽蒨居住在刘德华名下的塞马西湖自置寓所居住。 1991年，朱丽蒨母亲去世。 [3] 2001年12月，刘德华接受电台访问时，公开朱丽蒨身份。 2008年初，刘德华和朱丽蒨在一个佛教盛会上联名做善事，而这个佛教盛会规定只有亲人才可以联名，因此朱丽蒨也被坊间视为"刘太太"。', 'url': 'https://baike.baidu.com/item/%E6%9C%B1%E4%B8%BD%E8%92%A8/63815079'}, {'title': '朱丽倩演过的电视剧,朱丽倩演过的电影_明星_电视猫', 'content': '朱丽倩 15 人喜欢 英文名 Carol 生日 1966-04-06 出生地 大马槟城 身高 165公分 国家或地区 马来西亚 星座 白羊座 资料 影视作品 0 朱丽倩电视剧作品 朱丽倩电影作品 ...', 'url': 'https://www.tvmao.com/star/Flg=/works'}, {'title': '普罗米修斯(电影)', 'content': '2 weeks ago - 《普罗米修斯》的前期...景像。电影的制作过程被高度保密，其故事细节也是被“包裹得密不透风”。雷德利·斯科特决心要对剧情保密，他要求演员签订条款，防止他们泄露故事细节，演员只能在斯科特的制作办公室里看剧本，并且还...', 'url': 'https://zh.wikipedia.org/zh-hans/普罗米修斯_(电

{'input': '刘德华的老婆演过电影吗？',
 'chat_history': [HumanMessage(content='你好，我是老k。', additional_kwargs={}, response_metadata={}),
  AIMessage(content='你好，老k，很高兴认识你！', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='刘德华的老婆是谁？', additional_kwargs={}, response_metadata={}),
  AIMessage(content='刘德华的老婆是朱丽倩。', additional_kwargs={}, response_metadata={})],
 'output': '是的，朱丽倩演过电影和电视剧。'}